# 1. Giới thiệu

# 2. Word2vec

## 2.1. Giới thiệu

Word2vec là một kỹ thuật được giới thiệu bởi Tomas Mikolov vào năm 2013 để học word embeddings từ một lượng lớn dữ liệu văn bản. Word2vec không cần dữ liệu đã được gắn nhãn (như phân loại từ); thay vào đó, nó sử dụng self-supervised learning, nghĩa là nó tự học từ chính dữ liệu bằng cách dự đoán một phần dữ liệu dựa trên phần khác.

Word2vec có hai mô hình chính:
- **Skip-gram**: Skip-gram dự đoán các từ ngữ cảnh dựa trên từ mục tiêu. Với từ "sits", mô hình cố gắng dự đoán các từ như "the", "cat", "on", "the", "mat".
- **Continuous Bag of Words (CBOW)**: Ngược lại với Skip-gram, dự đoán một từ mục tiêu dựa trên các từ ngữ cảnh xung quanh nó. Ví dụ, với câu "the cat sits on the mat", CBOW lấy các từ "the", "cat", "on", "the", "mat" để dự đoán từ "sits".

Dưới đây là bảng so sánh nhanh giữa hai mô hình:

|Mô hình |	Mục tiêu |	Ví dụ|
|--------|-----------|-------|
|Skip-gram|	Dự đoán context từ center word|	Từ "ăn" dự đoán "tôi" và "táo".|
|Continuous Bag of Words (CBOW)|	Dự đoán center word từ context|	Từ "tôi" và "táo" dự đoán "ăn".|

<div style="text-align: center;">
    <div style="display: inline-block;">
        <p style="font-style: italic; color: gray; text-align: center; margin: 4px 0 0 0;">Bảng 1: So sánh Skip-gram và CBOW.</p>
    </div>
</div>

<div style="text-align: center;">
    <div style="display: inline-block;">
        <img src="https://github.com/hcmut-aa-math/word-embedding/blob/master/assets/pic3-cbow-and-skipgram.png?raw=true" style="width: 600px; height: auto;" />
        <p style="font-style: italic; color: gray; text-align: center; margin: 4px 0 0 0;">
            Hình 3: Kiến trúc CBOW dự đoán từ hiện tại dựa trên ngữ cảnh, còn Skip-gram dự đoán các từ xung quanh dựa vào từ hiện tại.
        </p>
    </div>
</div>

Cả hai mô hình đều dựa trên các công thức toán học để tính xác suất điều kiện, sử dụng hàm softmax và dot product giữa các vector từ. Dưới đây là phân tích chi tiết từng công thức.

## 2.2. Skip-gram
### 2.2.1 Định nghĩa
Mô hình skip-gam giả định rằng một từ có thể được sử dụng để sinh ra các từ xung quanh nó trong một chuỗi văn bản. Ví dụ, giả sử chuỗi văn bản là “the”, “man”, “loves”, “his” và “son”. Ta sử dụng “loves” làm từ đích trung tâm và đặt kích thước cửa sổ ngữ cảnh bằng 2. Như mô tả trong Hình 2, với từ đích trung tâm “loves”, mô hình skip-gram quan tâm đến xác suất có điều kiện sinh ra các từ ngữ cảnh (“the”, “man”, “his” và “son”) nằm trong khoảng cách không quá 2 từ:

$$
P(\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}\mid\textrm{"loves"}).
$$

Ta giả định rằng, với từ đích trung tâm cho trước, các từ ngữ cảnh được sinh ra độc lập với nhau. Trong trường hợp này, công thức trên có thể được viết lại thành

$$
P(\textrm{"the"}\mid\textrm{"loves"})\cdot P(\textrm{"man"}\mid\textrm{"loves"})\cdot P(\textrm{"his"}\mid\textrm{"loves"})\cdot P(\textrm{"son"}\mid\textrm{"loves"}).
$$


<div style="text-align: center;">
    <div style="display: inline-block;">
        <img src="https://github.com/hcmut-aa-math/word-embedding/blob/master/assets/pic4-skipgram.png?raw=true" style="width: 600px; height: auto;" />
        <p style="font-style: italic; color: gray; text-align: center; margin: 4px 0 0 0;">
            Hình 4: Mô hình skip-gram quan tâm đến xác suất có điều kiện sinh ra các từ ngữ cảnh với một từ đích trung tâm cho trước..
        </p>
    </div>
</div>


Trong mô hình skip-gam, mỗi từ được biểu diễn bằng hai vector $d-\text{chiều}$ (một dùng khi từ $w$ là từ ngữ cảnh, một dùng khi từ $w$ là từ trung tâm) để tính xác suất có điều kiện. Giả sử chỉ số của một từ trong từ điển là $i$, vector của từ được biểu diễn là $\mathbf{v}_i\in\mathbb{R}^d$ khi từ này là từ đích trung tâm và là $\mathbf{u}_i\in\mathbb{R}^d$ khi từ này là một từ ngữ cảnh. Gọi $c$ và $o$ lần lượt là chỉ số của từ đích trung tâm $w_c$ và từ ngữ cảnh $w_o$ trong từ điển. Có thể thu được xác suất có điều kiện sinh ra từ ngữ cảnh cho một từ đích trung tâm cho trước bằng phép toán softmax trên tích vô hướng của vector:

$$
P(w_o \mid w_c) = \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)},
$$

Trong đó:
- $w_c$: Từ trung tâm (center word), ví dụ "cat" trong câu "the cat sat on mat".
- $w_o$: Từ ngữ cảnh (output word), ví dụ "sat" trong cùng câu.
- $\mathbf{v}_c \in \mathbb{R}^d$: Vector biểu diễn của từ trung tâm $w_c$ (input vector).
- $\mathbf{u}_o \in \mathbb{R}^d$: Vector biểu diễn của từ ngữ cảnh $w_o$ (output vector).
- $\mathcal{V}$: Tập hợp tất cả từ trong từ điển (vocabulary). Tập chỉ số trong bộ từ vựng là $\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$.
- $\mathbf{u}_i \in \mathbb{R}^d$: Vector biểu diễn của từ $i$ trong từ điển.
- $\exp$: Hàm mũ, $\exp(x) = e^x$
- $\mathbf{u}_o^\top \mathbf{v}_c$: Tích vô hướng (dot product) giữa vector $\mathbf{u}_o$ và $\mathbf{v}_c$, đo lường mức độ tương đồng giữa từ trung tâm và từ ngữ cảnh.
- $\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)$: Tổng chuẩn hóa (normalization term) giữa các điểm tương đồng mũ giữa từ trung tâm $w_c$ trên toàn bộ từ điển $\mathcal{V}$, đảm bảo các xác suất cộng lại bằng 1.

**Ví dụ**:

Giả sử d=2 (vector 2 chiều), từ trung tâm $w_c = \text{"cat"}$, từ ngữ cảnh $w_o = \text{"pet"}$, và từ không liên quan $w_i = \text{"table"}$.

$\mathbf{v}_{\text{cat}} = [1, 0]$, $\mathbf{u}_{\text{pet}} = [0.7, 0.4]$, $\mathbf{u}_{\text{table}} = [0.1, -0.5]$.

Tích vô hướng:
  - $\mathbf{u}_{\text{pet}}^\top \mathbf{v}_{\text{cat}} = 0.7 \cdot 1 + 0.4 \cdot 0 = 0.7$
  - $\mathbf{u}_{\text{table}}^\top \mathbf{v}_{\text{cat}} = 0.1 \cdot 1 + (-0.5) \cdot 0 = 0.1$

Hàm mũ:
  - $\exp(0.7) \approx 2.0138$
  - $\exp(0.1) \approx 1.1052$

### 2.2.2. Hàm hợp lý (likelihood)
Giả sử ta có một chuỗi văn bản với độ dài $T$, trong đó từ tại vị trí $t$ được ký hiệu là $w^{(t)}$. Mô hình skip-gram giả định rằng:
- Các từ ngữ cảnh trong cửa sổ kích thước $m$ (bao gồm $m$ từ bên trái và $m$ từ bên phải của từ trung tâm) được sinh ra **độc lập** với nhau, khi biết từ đích trung tâm.
- Hàm hợp lý (likelihood) của mô hình là xác suất kết hợp để sinh ra tất cả các từ ngữ cảnh cho mọi từ trung tâm trong chuỗi văn bản.

Hàm hợp lý được định nghĩa như sau:

$$
L = \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}),
$$

Nếu $t+j < 1$ hoặc $t+j > T$ (vị trí ngoài chuỗi văn bản), các số hạng này bị bỏ qua. Ví dụ, với $t=1$ và $m=2$, các vị trí $j=-2, -1$ sẽ không tồn tại và được bỏ qua.

Ý nghĩa của hàm hợp lý:
- Hàm $L$ biểu thị xác suất tổng quát để mô hình skip-gram sinh ra toàn bộ các cặp từ trung tâm và từ ngữ cảnh trong tập dữ liệu.
- Mục tiêu của MLE là tìm các tham số ($\mathbf{v}_i$, $\mathbf{u}_i$) sao cho $L$ đạt giá trị lớn nhất, tức là mô hình dự đoán các từ ngữ cảnh chính xác nhất có thể.


### 2.2.3. Trainning
Mục tiêu của skip-gram là cực đại hóa hàm hợp lý (Maximum Likelihood Estimation - MLE), tức là tìm các vector $\mathbf{v}_i$ và $\mathbf{u}_i$ để cho $L$ đạt giá trị lớn nhất. Hay mô hình có thể dự đoán chính xác các từ ngữ cảnh dựa trên từ trung tâm.

Trong thực tế, vì $L$ là một tích của nhiều xác suất nhỏ, ta thường làm việc với log-likelihood để tránh vấn đề số học (numerical underflow):
$$
\log L = \sum_{t=1}^T \sum_{-m \leq j \leq m, j \neq 0} \log P(w^{(t+j)} \mid w^{(t)}).
$$

Hàm mất mát được định nghĩa là phủ định của log-likelihood:
$$
J = -\log L = -\sum_{t=1}^T \sum_{-m \leq j \leq m, j \neq 0} \log P(w^{(t+j)} \mid w^{(t)}).
$$

Hàm mất mát này là tổng của logarit xác suất có điều kiện trên tất cả các cặp từ trung tâm và từ ngữ cảnh trong tập huấn luyện. Việc giảm thiểu hàm mất mát này giúp mô hình cải thiện khả năng dự đoán.

Cực đại hóa $\log L$ tương đương với giảm thiểu $J$, để huấn luyện, ta sử dụng [**Stochastic Gradient Descent (SGD)**](https://en.wikipedia.org/wiki/Stochastic_gradient_descent). Trong mỗi vòng lặp, ta lấy mẫu ngẫu nhiên một chuỗi con nhỏ hơn (mini-batch) từ tập dữ liệu, tính toán hàm mất mát cho chuỗi con đó, sau đó tính gradient để cập nhật các tham số mô hình.
#### 2.2.3.1. Tính toán gradient: Điểm then chốt của huấn luyện

Để cập nhật các vector $\mathbf{v}_c$ và $\mathbf{u}_o$, ta cần tính gradient của hàm mất mát đối với các tham số. Điểm mấu chốt là tính gradient của logarit xác suất có điều kiện $\log P(w_o \mid w_c)$, trong đó $w_c$ là từ đích trung tâm và $w_o$ là từ ngữ cảnh.

Theo định nghĩa trong mô hình skip-gram, xác suất có điều kiện được tính như sau:
$$
P(w_o \mid w_c) = \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)},
$$
và logarit của nó là:
$$
\begin{align*}
\log P(w_o \mid w_c)
&= \log \frac{\exp(\mathbf{u}_o^\top \mathbf{v}_c)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)} \\
&= \log \exp(\mathbf{u}_o^\top \mathbf{v}_c) - \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right) \\
&= \mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right)
\end{align*}
$$


#### 2.2.3.2. Chứng minh gradient theo $\mathbf{v}_c$

Ta cần tính đạo hàm của $\log P(w_o \mid w_c)$ theo $\mathbf{v}_c$:
$$
\begin{align*}
\frac{\partial}{\partial \mathbf{v}_c}\log P(w_o \mid w_c) 
&= \frac{\partial}{\partial \mathbf{v}_c} \left( \mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right) \right) \\
&= \frac{\partial}{\partial \mathbf{v}_c} \mathbf{u}_o^\top \mathbf{v}_c - \frac{\partial}{\partial \mathbf{v}_c} \log\left(\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)\right) \\ 
&= \mathbf{u}_o - \frac{1}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)} \cdot \sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^\top \mathbf{v}_c) \mathbf{u}_j
\end{align*}
$$

Nhận thấy:
$$
\frac{\exp(\mathbf{u}_j^\top \mathbf{v}_c)}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)} = P(w_j \mid w_c),
$$
nên:
$$
\frac{1}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)} \cdot \sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^\top \mathbf{v}_c) \mathbf{u}_j = \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.
$$
Do đó, gradient tổng quát là:
$$
\frac{\partial \log P(w_o \mid w_c)}{\partial \mathbf{v}_c} = \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.
$$
**Ý nghĩa**: Gradient này bao gồm hai phần:
- $\mathbf{u}_o$: Vector từ ngữ cảnh thực tế, đại diện cho từ $w_o$ mà mô hình cần dự đoán.
- $\sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j$: Kỳ vọng của vector từ ngữ cảnh trên toàn bộ từ điển, dựa trên xác suất dự đoán.

Gradient này được sử dụng để cập nhật $\mathbf{v}_c$ trong SGD theo công thức:
$$
\mathbf{v}_c \leftarrow \mathbf{v}_c + \eta \left( \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j \right),
$$
với $\eta$ là tốc độ học (learning rate).
#### 2.2.3.3. Kết quả sau huấn luyện

Sau khi huấn luyện, với mỗi từ $w_i$ trong từ điển (có chỉ số $i$), ta thu được hai vector:

- $\mathbf{v}_i$: Vector từ đích trung tâm.
- $\mathbf{u}_i$: Vector từ ngữ cảnh.

Trong các ứng dụng NLP (như phân loại văn bản, phân tích cảm xúc), **vector từ đích trung tâm $\mathbf{v}_i$** thường được sử dụng làm biểu diễn chính cho từ $w_i$, vì nó được tối ưu hóa để dự đoán ngữ cảnh xung quanh.

# 3. Bài tập

## Câu 1:
Độ phức tạp tính toán của mỗi gradient là bao nhiêu? Nếu từ điển chứa một lượng lớn các từ, điều này sẽ gây ra vấn đề gì?

### 1.1. Độ phức tạp tính toán của mỗi gradient

Nhắc lại công thức tính gradient của hàm mất mát theo $\mathbf{v}_c$ trong mô hình Skip-gram (15.1.8):

$$
\frac{\partial \textrm{log}\, P(w_o \mid w_c)}{\partial \mathbf{v}_c} = \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j
$$

Nhắc lại công thức tính gradient của hàm mất mát theo $\mathbf{v}_{o_i}$ trong mô hình CBOW (15.1.15):

$$
\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}} = \frac{1}{2m}\left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j \right)
$$


- Dễ thấy, đối với cả 2 mô hình, để tính được gradient ta phải tính xác suất có điều kiện cho mọi từ $w_j$ có trong từ điển $\mathcal{V}$ với một từ cho trước.
- Do đó, độ phức tạp tính toán của mỗi gradient trong cả 2 mô hình đều là $O(|\mathcal{V}| \cdot d)$, với $d$ là số chiều của vector embedding và $\mathcal{V}$ là từ điển


### 1.2. Vấn đề khi từ điển chứa một lượng lớn các từ

- Từ độ phức tạp trên ta thấy, mỗi lần huấn luyện cho một cặp từ, ta đều phải tính xác suất cho tất cả các từ trong từ điển
- Điều này rất tốn thời gian, dẫn đến huấn luyện rất chậm hội tụ, không khả thi với dữ liệu thực tế.

**Giải pháp:** Dùng kỹ thuật Negative Sampling (xem mục 15.2.1)

### Ý tưởng: Nhận diện và tạo cụm từ (phrases) trước khi huấn luyện Word2Vec

Trước khi huấn luyện Word2Vec, ta xác định ra các cụm từ phổ biến trong một ngữ cảnh nhưng ít phổ biến trong các ngữ cảnh khác, như: `"new york"`, `"san francisco"`... (trong khi cụm `"this is"` thì xuất hiện trong rất nhiều ngữ cảnh), sau đó nối chúng lại và coi chúng là một từ riêng trong từ điển khi huấn luyện embedding.

---

## Phương pháp phát hiện cụm từ: Dựa trên thống kê đồng xuất hiện
Có nhiều phương pháp phát hiện cụm từ đã được nghiên cứu trước đó, nhưng chúng không nằm trong phạm vi nghiên cứu của bài báo nên nhóm tác giả chỉ sử dụng phương pháp đơn giản và đủ hiệu quả.

Ta kiểm tra xem cặp từ $w_i, w_j$ có nên được nối thành một cụm hay không bằng công thức sau:

$$
\text{score}(w_i, w_j) = \frac{C(w_i w_j) - \delta}{C(w_i) \cdot C(w_j)}
$$

Trong đó:
- $C(w_i)$, $C(w_j)$: tần suất của từng từ riêng lẻ,
- $C(w_i w_j)$: tần suất của cụm 2 từ xuất hiện liên tiếp,
- $\delta$: một hằng số để tránh nối những cụm có tần suất thấp (thường là 5).

Đồng thời đặt ra một ngưỡng cố định. Nếu score vượt ngưỡng này, ta xem $w_i w_j$ là cụm cố định.

> **Ví dụ 1**:  
> Nếu `"new"` xuất hiện 5000 lần, `"york"` 3000 lần, `"new york"` xuất hiện 2800 lần, thì:
>
> $\text{score}(\text{"new"}, \text{"york"}) = \frac{2800 - 5}{5000 \cdot 3000} \approx 0.000186$
> 
> Nếu ngưỡng là 0.0001 → `"new york"` được giữ lại thành `"new_york"`.

> **Ví dụ 2**:  
> Nếu `"this"` xuất hiện 10000 lần, `"is"` 9500 lần, `"this is"` xuất hiện 5000 lần, thì:
>
> $\text{score}(\text{"this"}, \text{"is"}) = \frac{5000 - 5}{10000 \cdot 9500} \approx 0.0000525789474$
> 
> Nếu ngưỡng là 0.0001 → `"this is"` không được nối lại thành cụm từ

### 3.1. Mối quan hệ giữa tích vô hướng của hai vector từ và độ tương tự cô-sin

Nhắc lại công thức tính vô hướng trong trường hợp 2 vector từ:

$$
\boldsymbol{u}_{w_o}^\top \boldsymbol{v}_{w_c} = \|\boldsymbol{u}_{w_o}\| \cdot \|\boldsymbol{v}_{w_c}\| \cdot \cos(\theta)
$$

Trong đó:
- $\theta$: góc giữa hai vector.
- $\cos(\theta)$: **độ tương tự cosine** giữa hai vector, có miền giá trị từ -1 (khi 2 vector ngược hướng) đến 1 (khi 2 vector cùng hướng) và bằng 0 khi 2 vector vuông góc

👉 **Tích vô hướng càng lớn (theo chiều dương)**, nghĩa là:
- Hai vector càng **cùng hướng** (cosine gần 1),
- Hoặc có **độ dài lớn** (cũng ảnh hưởng, nhưng không mang nghĩa ngữ nghĩa nhiều bằng góc).



### 3.2. Vì sao các từ có nghĩa gần nhau thường có độ tương tự cosine cao?


Nhắc lại công thức của Skip-gram: Với cặp từ $(w_c, w_o)$, ta tối đa hóa xác suất sau:

$$
P(w_o | w_c) = \frac{\exp(\boldsymbol{u}_{w_o}^\top \boldsymbol{v}_{w_c})}{\sum_{w \in V} \exp(\boldsymbol{u}_w^\top \boldsymbol{v}_{w_c})}
$$

Quan sát công thức này ta thấy: Nếu hai từ hay cùng xuất hiện, tức là xác suất $P$ ở trên lớn, thì tích vô hướng $\boldsymbol{u}_{w_o}^\top \boldsymbol{v}_{w_c}$ cũng lớn.

Mặt khác, ta huấn luyện Skip-gram bằng cách cực đại hoá hợp lý xác suất $P$ đó. Do vậy, mô hình đã được huấn luyện sao cho làm tăng tích vô hướng của 2 vector từ tương tự nhau, từ đó làm tăng độ tương tự cosine.


**Tóm lại:**
- Trong mô hình Skip-Gram của Word2Vec, xác suất dự đoán từ phụ thuộc vào tích vô hướng giữa hai vector từ, mà tích vô hướng lại phụ thuộc trực tiếp vào độ tương tự cosine.
- Khi các từ có nghĩa gần nhau, chúng thường xuất hiện trong những ngữ cảnh giống nhau, khiến các vector của chúng hướng gần nhau trong không gian → độ tương tự cosine cao.